# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 25 2022 1:14PM,249359,10,0086109309,ISDIN Corporation,Released
1,Oct 25 2022 1:14PM,249359,10,0086109310,ISDIN Corporation,Released
2,Oct 25 2022 1:14PM,249359,10,0086109570,ISDIN Corporation,Released
3,Oct 25 2022 1:14PM,249359,10,0086109569,ISDIN Corporation,Released
4,Oct 25 2022 1:14PM,249359,10,0086109573,ISDIN Corporation,Released
5,Oct 25 2022 1:14PM,249359,10,0086109574,ISDIN Corporation,Released
6,Oct 25 2022 1:14PM,249359,10,0086109575,ISDIN Corporation,Released
7,Oct 25 2022 1:14PM,249359,10,0086109571,ISDIN Corporation,Released
8,Oct 25 2022 1:14PM,249359,10,0086109672,ISDIN Corporation,Released
9,Oct 25 2022 1:14PM,249359,10,0086109686,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,249356,Executing,5
22,249356,Released,13
23,249357,Released,1
24,249359,Released,29
25,249360,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249355,NaN,4.0,NaN
249356,NaN,5.0,13.0
249357,NaN,NaN,1.0
249359,NaN,NaN,29.0
249360,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0.0,1.0,0.0
249293,41.0,14.0,7.0
249299,0.0,14.0,29.0
249301,0.0,0.0,1.0
249302,0.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0,1,0
249293,41,14,7
249299,0,14,29
249301,0,0,1
249302,0,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,0,1,0
1,249293,41,14,7
2,249299,0,14,29
3,249301,0,0,1
4,249302,0,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,,1,
1,249293,41,14,7
2,249299,,14,29
3,249301,,,1
4,249302,,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation
29,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated"
30,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation
37,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated"
38,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated"
39,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated"
40,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd"
58,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc."
59,Oct 25 2022 1:02PM,249355,10,Methapharm-G
62,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation,,,29
1,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated",,,1
2,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation,,,7
3,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",,,1
4,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",,,1
5,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",,,1
6,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",,5,13
7,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc.",,4,
8,Oct 25 2022 1:02PM,249355,10,Methapharm-G,,4,
9,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation,29,,
1,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated",1,,
2,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation,7,,
3,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1,,
4,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1,,
5,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",1,,
6,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",13,5,
7,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc.",,4,
8,Oct 25 2022 1:02PM,249355,10,Methapharm-G,,4,
9,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation,29,,
1,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated",1,,
2,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation,7,,
3,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1,,
4,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation,29.0,NaN,NaN
1,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation,7.0,NaN,NaN
3,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:14PM,249359,10,ISDIN Corporation,29.0,0.0,0.0
1,Oct 25 2022 1:13PM,249357,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Oct 25 2022 1:12PM,249360,10,ISDIN Corporation,7.0,0.0,0.0
3,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2742849,43.0,30.0,14.0
12,249279,0.0,1.0,0.0
15,747894,43.0,28.0,41.0
19,748049,15.0,5.0,0.0
20,249315,0.0,1.0,0.0
21,249301,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2742849,43.0,30.0,14.0
1,12,249279,0.0,1.0,0.0
2,15,747894,43.0,28.0,41.0
3,19,748049,15.0,5.0,0.0
4,20,249315,0.0,1.0,0.0
5,21,249301,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,43.0,30.0,14.0
1,12,0.0,1.0,0.0
2,15,43.0,28.0,41.0
3,19,15.0,5.0,0.0
4,20,0.0,1.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,43.0
1,12,Released,0.0
2,15,Released,43.0
3,19,Released,15.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,14.0,0.0,41.0,0.0,0.0,0.0
Executing,30.0,1.0,28.0,5.0,1.0,0.0
Released,43.0,0.0,43.0,15.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,14.0,0.0,41.0,0.0,0.0,0.0
1,Executing,30.0,1.0,28.0,5.0,1.0,0.0
2,Released,43.0,0.0,43.0,15.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,14.0,0.0,41.0,0.0,0.0,0.0
1,Executing,30.0,1.0,28.0,5.0,1.0,0.0
2,Released,43.0,0.0,43.0,15.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()